In [21]:
import os
import pandas as pd
files = pd.read_csv('ef.csv').ix[:,0].tolist()
tmp_files = pd.read_csv('ef.csv')
tmp_files.columns = ['name']

/home/fattahov.ao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
p1 = pd.read_csv('preds/1_train.csv',header=None)
p2 = pd.read_csv('preds/2_train.csv',header=None)
p3 = pd.read_csv('preds/3_train.csv',header=None)
p4 = pd.read_csv('preds/4_train.csv',header=None)
p5 = pd.read_csv('preds/5_train.csv',header=None)
p6 = pd.read_csv('preds/6_train.csv',header=None)

p3 = p3[:6905]
p4 = p4[:6905]
p5 = p5[:6905]
p6 = p6[:6905]

In [23]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [24]:
# Стратифицированное разбиение на фолды
def skf(a):
    n = len(a)
    b = [0, 0]
    for i in a:
        b[i] += 1
    bb = []
    for i in range(5):
        bb += [[0, 0]]
    for i in range(2):
        for j in range(b[i] % 5):
            bb[j][i] = b[i] // 5 + 1
        for j in range(b[i] % 5, 5):
            bb[j][i] = b[i] // 5;
 
    cur = [0, 0]
    c = []
    for i in range(5):
        c += [[0, 0]]
    ans = []
    for i in range(5):
        ans += [[[], []]]
    for i in range(n):
        if c[cur[a[i]]][a[i]] < bb[cur[a[i]]][a[i]]:
            for j in range(5):
                if (cur[a[i]] == j):
                    ans[j][0].append(i)
                else:
                    ans[j][1].append(i)
            c[cur[a[i]]][a[i]] += 1
            if (c[cur[a[i]]][a[i]] == bb[cur[a[i]]][a[i]]):
                cur[a[i]] += 1;
    return ans

In [25]:
import numpy as np
classes = []
y = []
for i in files:
    if 'indoor' in i:
        classes += [[0, 1]]
        y += [1]
    else:
        classes += [[1, 0]]
        y += [0]
classes = np.array(classes)
y = np.array(y)

In [26]:
preds = pd.concat([p1, p2, p3, p4, p5, p6], axis=1)
preds.columns = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6']

In [7]:
preds[:20]

,p1,p2,p3,p4,p5,p6
0,0.726661,0.872270,0.992406,0.977511,0.999973,0.908865
1,0.999976,0.999976,0.999764,0.999903,0.999999,0.999893
2,0.201203,0.109141,0.060366,0.118175,0.004036,0.031256
3,0.938829,0.831030,0.868279,0.896412,0.997449,0.897891
4,0.425932,0.145346,0.860063,0.231303,0.001202,0.538335
5,0.657997,0.246363,0.677175,0.359080,0.810799,0.584259
6,0.990974,0.952459,0.973173,0.962963,0.999762,0.959905
7,0.993630,0.995189,0.992105,0.994251,0.999830,0.984658
8,0.364095,0.180749,0.683647,0.457735,0.004145,0.138814
9,0.999866,0.999968,0.998441,0.999961,0.999910,0.999980


In [8]:
import sys
import numpy as np
from scipy.stats import logistic
 
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def logloss(true_label, predicted, eps=1e-15):
    p = np.clip(predicted, eps, 1 - eps)
    if true_label == 1:
        return -np.log(p)
    else:
        return -np.log(1 - p)
    
def log_loss(y_true, y_pred):
    los = 0
    for i in range(len(y_true)):


        los += logloss(y_true[i][1], y_pred[i][1])
            
    return los / len(y_true)    
    
def log_loss1(y_true, y_pred):
    los = 0
    for i in range(len(y_true)):

        los += logloss(y_true[i], y_pred[i])
            
    return los / len(y_true)     

    
def sigmoid(x):
    return logistic.cdf(x)
"""
def softmax(x):
    e = np.exp(x - np.max(x))  # prevent overflow
    if e.ndim == 1:
        return e / np.sum(e, axis=0)
    else:  
        return e / np.array([np.sum(e, axis=1)]).T  # ndim = 2
"""

def softmax(X, copy=True):

    if copy:
        X = np.copy(X)
    max_prob = np.max(X, axis=1).reshape((-1, 1))
    X -= max_prob
    np.exp(X, X)
    sum_prob = np.sum(X, axis=1).reshape((-1, 1))
    X /= sum_prob
    return X

class LogisticRegression_(object):
    def __init__(self, X, y, X_val, y_val):
        self.X = X
        self.y = y
        self.X_val = X_val
        self.y_val = y_val
        self.W = np.zeros((X.shape[1], y.shape[1]))  # initialize W 0
        self.b = np.zeros(y.shape[1])          # initialize bias 0
        self.losses_train = []
        self.losses_valid = []

        # self.params = [self.W, self.b]

    def fit(self, n_epochs=1000, lr=0.0001, L2_reg=0.001):
        for i in range(n_epochs):
            # p_y_given_x = sigmoid(numpy.dot(self.x, self.W) + self.b)
            p_y_given_x = softmax(np.dot(self.X, self.W) + self.b)
            d_y = self.y - p_y_given_x

            self.W += lr * np.dot(self.X.T, d_y) - lr * L2_reg * self.W
            self.b += lr * np.mean(d_y, axis=0)
            #print(softmax(np.dot(self.X, self.W) + self.b))

            #print(softmax(np.dot(self.X_val, self.W) + self.b))
            if i % 500 == 0:
                loss = log_loss(self.y, softmax(np.dot(self.X, self.W) + self.b))
                val_loss = log_loss(self.y_val, softmax(np.dot(self.X_val, self.W) + self.b))
                #loss = self.cross_entropy(val=False)
                #val_loss = self.cross_entropy(val=True)
            
                self.losses_train += [loss]
                self.losses_valid += [val_loss]
                print('Train_loss=', loss,' Val_loss=', val_loss)

    def cross_entropy(self, val=False):
        if val:
            X = self.X_val
            y = self.y_val
        else:
            X = self.X
            y = self.y
            
        # sigmoid_activation = sigmoid(numpy.dot(self.x, self.W) + self.b)
        sigmoid_activation = softmax(np.dot(X, self.W) + self.b)

        cross_entropy = - np.mean(np.sum(y * np.log(sigmoid_activation) + 
                                               (1 - y) * np.log(1 - sigmoid_activation),axis=1))
        return cross_entropy
    
    def predict_proba(self, X):
        # return sigmoid(np.dot(x, self.W) + self.b)
        return softmax(np.dot(X, self.W) + self.b)

    def predict(self, X):
        # return sigmoid(np.dot(x, self.W) + self.b)
        preds = []
        probas = softmax(np.dot(X, self.W) + self.b)
        for i in probas:
            preds += [i == i.max()]
        return np.array(preds).astype(int)
    
    

In [9]:
tmp = skf(y)
    
folds = []    
for i in tmp:
    folds += [[i[1], i[0]]]

In [10]:
tr = np.array(preds)

In [11]:
tr.shape

(6905, 6)

In [12]:
pred_train = np.zeros(len(tr))
est = []
f_ll,f_rmse = [],[]

for itr, ite in folds:
    ypred = []
    tmp_tr = tr[itr]
    tmp_classes = classes[itr]
    classifier = LogisticRegression_(tmp_tr, tmp_classes, tr[ite], classes[ite])
    classifier.fit(n_epochs=8100, lr=0.002, L2_reg=0.001)

    ypred = classifier.predict(tr[ite])[:, 1]

    
    pred_train[ite] = ypred 

    print(log_loss1(y[ite], pred_train[ite]))
    print(rmse(y[ite], pred_train[ite]))
    
    f_ll.append(log_loss1(y[ite], pred_train[ite]))
    f_rmse.append(rmse(y[ite], pred_train[ite]))

    est.append(classifier)
    
print('logreg: {:.3f} +- {:.3f}'.format(np.mean(f_ll), np.std(f_ll)))
print('rmse: {:.3f} +- {:.3f}'.format(np.mean(f_rmse), np.std(f_rmse)))

Train_loss= 1.73871491486  Val_loss= 1.78600457519
Train_loss= 0.25601013909  Val_loss= 0.257772060073
Train_loss= 0.235745704159  Val_loss= 0.238628413829
Train_loss= 0.218778307295  Val_loss= 0.222546240253
Train_loss= 0.204472773512  Val_loss= 0.209035265711
Train_loss= 0.192337855468  Val_loss= 0.197627604378
Train_loss= 0.181975293019  Val_loss= 0.187932601015
Train_loss= 0.17306429986  Val_loss= 0.179634769983
Train_loss= 0.165347531793  Val_loss= 0.172481587377
Train_loss= 0.158618641242  Val_loss= 0.16627118567
Train_loss= 0.152711814181  Val_loss= 0.160841836905
Train_loss= 0.147493276375  Val_loss= 0.156063412496
Train_loss= 0.14285453786  Val_loss= 0.151830599358
Train_loss= 0.138707077525  Val_loss= 0.148057558402
Train_loss= 0.134978178204  Val_loss= 0.14467371252
Train_loss= 0.131607661652  Val_loss= 0.141620373805
Train_loss= 0.128545324582  Val_loss= 0.138847920889
1.19962869325
0.186365962744
Train_loss= 1.75912508089  Val_loss= 1.68199724049
Train_loss= 0.261402056087

In [13]:
p1 = pd.read_csv('preds/1.csv')
p2 = pd.read_csv('preds/2.csv').ix[:,1]
p3 = pd.read_csv('preds/3.csv').ix[:,1]
p4 = pd.read_csv('preds/4.csv').ix[:,1]
p5 = pd.read_csv('preds/5.csv').ix[:,1]
p6 = pd.read_csv('preds/6.csv').ix[:,1]

/home/fattahov.ao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/fattahov.ao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/home/fattahov.ao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-depre

In [14]:
preds_test = pd.concat([p1, p2, p3, p4, p5, p6], axis=1)
#preds.columns = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6']

In [15]:
idx = preds_test.id
del preds_test['id']

In [16]:
preds_test[:10]

,res,res,res,res,res,res
0,0.002582,0.003858,0.042007,0.004032,0.059628,0.010672
1,0.873999,0.979739,0.987742,0.978227,0.999935,0.945789
2,0.729980,0.755818,0.828963,0.822823,0.996606,0.527153
3,0.620298,0.504142,0.742050,0.437726,0.710074,0.548276
4,0.999550,0.999407,0.981607,0.994913,0.999952,0.972804
5,0.173062,0.314176,0.275407,0.386468,0.073598,0.323531
6,0.912085,0.862033,0.976372,0.913806,0.999524,0.965701
7,0.999951,0.999995,0.999862,0.999959,0.999993,0.999803
8,0.986881,0.994741,0.987167,0.996171,0.999834,0.962648
9,0.039438,0.012841,0.118860,0.102356,0.600754,0.185040


In [17]:
te = np.array(preds_test)

In [18]:
# Усреднение 
res = est[0].predict_proba(te)

for i in range(1,5):
    res += est[i].predict_proba(te)

res = res / 5.0

In [19]:
tmp = pd.DataFrame(idx)
tmp.columns = ['id']
tmp['res'] = res[:,1]
tmp.to_csv('preds/st_188.csv', index=False)

In [19]:
tmp

,id,res
0,223,0.146623
1,1878,0.986377
2,409,0.986877
3,1993,0.857029
4,783,0.984741
5,900,0.282334
6,2508,0.981243
7,1848,0.984503
8,1285,0.985135
9,2889,0.722239
